<h3>Count of query terms aproximate number of results</h3>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import time

def resultados_por_ano_2(query_term):
    sites = 'publico.pt,sapo.pt,jn.pt,observador.pt,expresso.pt'

    resultados = []
    titulos = []
    for i in range(2010, 2023):
        data = [int(f"{i}0101"), int(f"{i}1231")]
        

        site= f"https://arquivo.pt/textsearch?q={query_term}&prettyPrint=false&from={data[0]}&to={data[1]}&siteSearch={sites}"
        json = requests.get(site).json()["response_items"]
        content = requests.get(site).json()
        resultados.append(content['estimated_nr_results'])
        # print(f"{query_term}, {i}, ok", end=" | ")
        #try:
        #    print(json[0]["linkToExtractedText"])
        #except:
        #    pass

    return resultados

df = pd.DataFrame({"anos": list(range(2010, 2023))})
# Query terms to search for
query_terms = [
    "Tesla", "carro elétrico", "Renault Zoe",
    "emissão zero",
    "mobilidade sustentável", "Tesla Model 3", "Tesla Model Y","poluição do ar" 
]
for term in query_terms:
    df[term] = resultados_por_ano_2(term)

df

,anos,Tesla,carro elétrico,Renault Zoe,emissão zero,mobilidade sustentável,Tesla Model 3,Tesla Model Y,poluição do ar
0,2010,0,0,0,0,0,0,0,0
1,2011,0,0,0,5,0,0,0,6
2,2012,1,0,0,11,0,0,0,7
3,2013,0,0,0,36,19,0,0,73
4,2014,78,47,0,103,9,0,0,452
5,2015,17580,6660,0,250,266,6432,6477,1992
6,2016,62888,14418,11128,2456,1468,25668,27148,16876
7,2017,113383,5858,28044,1518,1765,27161,31919,7688
8,2018,75352,20763,13389,1703,1883,29279,29280,4870
9,2019,42978,18578,7804,6156,8813,23268,23668,14241


<h3>Code for webscraping websites</h3>

In [5]:
# Extend the domain-to-CSS mapping to include both CSS classes and multiple HTML elements
domain_css_element_map = {
    "publico.pt": {"css_class": None, "html_elements": ["p","h1"]}, 
    "sapo.pt": {"css_class": None, "html_elements": ["p","h1"]},  
    "jn.pt": {"css_class": None, "html_elements": ["p", "h1"]},  # Extract both <p> and <h1> elements
    "observador.pt": {"css_class": None, "html_elements": ["p","h1"]},  
    "expresso.pt": {"css_class": None, "html_elements": ["p","h1"]},  
}

# Function to normalize the domain (remove 'www.')
def normalize_domain(domain):
    if domain.startswith("www."):
        return domain[4:]  # Remove 'www.' prefix
    return domain

# Function to extract text based on either a CSS class or multiple HTML elements
def extract_text_by_css_class_or_element(original_url, domain_css_element_map):
    try:
        # Parse the domain from the URL
        parsed_url = urlparse(original_url)
        domain = normalize_domain(parsed_url.netloc)  # Normalize the domain
        
        # Get the extraction rule for this domain from the mapping
        extraction_rule = domain_css_element_map.get(domain)
        if not extraction_rule:
            print(f"No extraction rule found for domain: {domain}")
            return None
        
        css_class = extraction_rule.get("css_class")
        html_elements = extraction_rule.get("html_elements")

        # Set a user-agent to mimic a browser request
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        # Initialize backoff_time
        backoff_time = 1  # Start with 1 second
        
        # Fetch the content of the original page
        response = requests.get(original_url, headers=headers)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')
            
            text_content = []
            
            # Extract text based on the CSS class if provided
            if css_class:
                elements = soup.find_all(class_=css_class)
                text_content = [element.get_text(strip=True) for element in elements]
            
            # Extract text based on the HTML elements if provided
            if html_elements:
                for element in html_elements:
                    elements = soup.find_all(element)
                    text_content.extend([elem.get_text(strip=True) for elem in elements])
            
            return text_content
        elif response.status_code == 429:
                # print(f"Too Many Requests (429): {original_url}, sleeping for {backoff_time} seconds")
                time.sleep(backoff_time)
                backoff_time = min(backoff_time * 2, 60)  # Increase wait time up to 60 seconds
        elif response.status_code == 404:
                # print(f"URL not found (404): {original_url}")
                return None
        else:
            # print(f"Failed to retrieve {original_url}: Status Code {response.status_code}")
            return None
    except Exception as e:
        # print(f"Error fetching {original_url}: {e}")
        return None

# Function to get results from Arquivo.pt API and then filter by text content
def resultados_por_ano(query_term, domain_css_element_map):
    sites = 'publico.pt,sapo.pt,jn.pt,expresso.pt,observador.pt'
    textos_filtrados = []
    
    for i in range(2010, 2023):  # Loop through each year
        data = [int(f"{i}0101"), int(f"{i}1231")]
        site = f"https://arquivo.pt/textsearch?q={query_term}&prettyPrint=false&from={data[0]}&to={data[1]}&siteSearch={sites}"
        
        try:
            response = requests.get(site)
            content = response.json()
            
            # For each result, extract text from the original URL
            for item in content['response_items']:
                original_url = item['originalURL']
                extracted_text = extract_text_by_css_class_or_element(original_url, domain_css_element_map)
                
                if extracted_text:
                    # combined_text = " ".join(extracted_text)  # Combine text into one string
                    # # Check if the query term appears in the combined text (case-insensitive)
                    # if query_term.lower() in combined_text.lower(): 
                        textos_filtrados.append({
                            "year": i,
                            "query_term": query_term,
                            "title": item['title'],
                            "url": original_url,
                            "extracted_text": extracted_text  # Store the combined text
                        })

        except Exception as e:
            print(f"Error processing year {i} for query {query_term}: {e}")
    
    return textos_filtrados

# Query terms to search for
query_terms = [
    "Tesla", "carro elétrico", "Renault Zoe",
    "emissão zero",
    "mobilidade sustentável", "Tesla Model 3", "Tesla Model Y","poluição do ar" 
]

# Dictionary to store filtered texts for each query
filtered_texts = []

# Loop through each query term and collect data
for term in query_terms:
    textos_filtrados = resultados_por_ano(term, domain_css_element_map)
    filtered_texts.extend(textos_filtrados)
    print(f"Finished processing {term}")

# Convert the filtered texts into a DataFrame
df_articles = pd.DataFrame(filtered_texts)

# Grouping by year and query term to count the number of articles per year for each term
df_article_counts = df_articles.groupby(["year", "query_term"]).size().unstack(fill_value=0)

# Display the article count DataFrame (one column per query term, one row per year)
print(df_article_counts)

# Save the articles DataFrame to CSV for future analysis if needed
df_articles.to_csv("extracted_articles.csv", index=False)

# Save the article counts to another CSV
df_article_counts.to_csv("article_counts_by_year.csv")

df_articles.head()


c:\Users\FF\miniconda3\envs\fcd\Lib\html\parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Finished processing Tesla
Finished processing carro elétrico
Finished processing Renault Zoe
Finished processing emissão zero
Finished processing mobilidade sustentável
Finished processing Tesla Model 3
Finished processing Tesla Model Y
Finished processing poluição do ar
query_term  Renault Zoe  Tesla  Tesla Model 3  Tesla Model Y  carro elétrico  \
year                                                                           
2011                  0      0              0              0               0   
2012                  0      1              0              0               0   
2013                  0      0              0              0               0   
2014                  0     49              0              0              22   
2015                  0     32             39             35              47   
2016                 50     48             48             45              48   
2017                 47     44             47             43              48   
2018    

,year,query_term,title,url,extracted_text
0,2012,Tesla,PUBLICO.PT,http://publico.pt/,[Os leitores são a força e a vida dos jornais....
1,2014,Tesla,A Tesla vai instalar postos de super carregame...,http://observador.pt/2014/09/11/tesla-vai-inst...,"[Rádio, Apoie o jornalismo independente e leia..."
2,2014,Tesla,,http://observador.pt/2014/09/11/tesla-vai-inst...,"[Rádio, Apoie o jornalismo independente e leia..."
3,2014,Tesla,Pedro Esteves - Observador,http://observador.pt/perfil/pesteves/,"[Rádio, Apoie o jornalismo independente e leia..."
4,2014,Tesla,Pedro Esteves - 2/10455 - Observador,http://observador.pt/perfil/pesteves/page/2/,"[Rádio, Apoie o jornalismo independente e leia..."
